# Some examples of how to load klayout technology definition

In [1]:
# Reload modules every time code is called. Set autoreload 0 to disable
%load_ext autoreload
%autoreload 2

In [2]:
import phidl.technology as tech
# from phidl import quickplot2 as qp

In [3]:
tech.TECHNOLOGY_PATH

'/Users/atait/Dropbox/Documents/gitProjects/layout-code/SOEN_PDK/klayout-dot-config/tech/OLMAC'

In [6]:
tech.tech_top()['technology']

{'name': 'OLMAC',
 'description': None,
 'dbu': '0.001',
 'base-path': None,
 'original-base-path': '~/.klayout/salt/soen_olmac_pdk/tech/OLMAC',
 'layer-properties_file': 'klayout_layers_OLMAC.lyp',
 'add-other-layers': 'true',
 'reader-options': {'common': {'create-other-layers': 'true',
   'layer-map': 'layer_map()',
   'enable-properties': 'true',
   'enable-text-objects': 'true'},
  'gds2': {'box-mode': '1',
   'allow-big-records': 'true',
   'allow-multi-xy-records': 'true'},
  'dxf': {'dbu': '0.001',
   'unit': '1',
   'text-scaling': '100',
   'circle-points': '100',
   'circle-accuracy': '0',
   'polyline-mode': '0',
   'render-texts-as-polygons': 'false',
   'keep-other-cells': 'false',
   'create-other-layers': 'true',
   'layer-map': 'layer_map()'},
  'cif': {'wire-mode': '0',
   'dbu': '0.001',
   'layer-map': 'layer_map()',
   'create-other-layers': 'true'},
  'lefdef': {'read-all-layers': 'true',
   'layer-map': 'layer_map()',
   'dbu': '0.001',
   'produce-net-names': 't

In [14]:
lys = tech.get_technology_by_name(1)
print(lys)

{'technology_name': 'OLMAC', 'dbu': '0.001', 'base_path': '/Users/atait/Dropbox/Documents/gitProjects/layout-code/SOEN_PDK/klayout-dot-config/tech/OLMAC', '1/0 - dp_p+ - (p+ doping)': Layer (name unnamed, GDS layer 1, GDS datatype 0, description None, color None), '2/0 - dp_n+ - (n+ doping)': Layer (name unnamed, GDS layer 2, GDS datatype 0, description None, color None), '3/0 - dp_p - (p doping)': Layer (name unnamed, GDS layer 3, GDS datatype 0, description None, color None), '4/0 - dp_n - (n doping)': Layer (name unnamed, GDS layer 4, GDS datatype 0, description None, color None), '5/0 - dp_e - (emitter doping)': Layer (name unnamed, GDS layer 5, GDS datatype 0, description None, color None), '11/0 - m1_nwpad - (nanowire wiring and pads)': Layer (name unnamed, GDS layer 11, GDS datatype 0, description None, color None), '11/99 - m1_nwgnd - (nanowire wiring and pads)': Layer (name unnamed, GDS layer 11, GDS datatype 99, description None, color None), '12/0 - m2_nw - (superconductor (

In [5]:
import SiEPIC


ModuleNotFoundError: No module named 'pya'

In [ ]:
# This sets a global variable. 
# It also verifies that that technology exists in a place that is visible to klayout
tech.set_technology_by_name('OLMAC')
print(tech.get_technology_name())  # OLMAC
tech.set_technology_by_dir('~/.klayout/salt/SOEN_PDK/tech/OLMAC')  # no verification
print(tech.get_technology_name())  # OLMAC

## Files of interest are `.xml` and `.lyt`, 
(and the `.lyp` contained in the `OLMAC.lyt`)
### We don't really care about `.lym`, `.lydrc`, `.xs` files

In [ ]:
# load in some properties generically

# Load in the layer properties, convert to a phidl format

In [ ]:
# load in some property categories in a more useful format in terms of keys

# Use a new layout function called waveguide (or something)
#  It does NOT take layers as an argument, it takes 'Strip'
#  It takes points and automatically rounds the corners
#  (eventually) it could auto-widen